### 금융정보방 게시판에 있는 글들 다 모아오기.
#### author : 편도훈



In [1]:
from selenium import webdriver
import selenium
import pandas as pd
import numpy as np
import datetime
import time
from tqdm import tqdm
from glob import glob
import datetime
import os
from multiprocessing.pool import ThreadPool

In [2]:
# 1개 회사의 정보와 폴더이름을 가지고 게시판에 들어가서 크롤링하는 function입니다.

def crawl_(company_info_lists,folder_name):

    # 크롬드라이버가 연결되어 있는 주소 저장
    chromedriver ='E:/big12/python-project/note/driver/chromedriver.exe'

    # 크롬드라이버 연결
    driver = webdriver.Chrome(chromedriver)
    print(company_info_lists)
    #업종 사이트 연결
    title=[]
    times=[]
    contents=[]
    address=[]
    #받아온 정보의 index 0 인 코드를 넣습니다. page는 아무렇게나 하셔도 상관 없습니다. but 저는 1번째 페이지만 해놨습니다.
    driver.get('https://finance.naver.com/item/board.nhn?code={code}&page={page}'.format(code=company_info_lists[0],page=1))

    table = driver.find_element_by_class_name('type2')

    #게시글 제목 찾기
    title.extend([x.text.replace(","," ") for x in table.find_elements_by_tag_name('a')]) 

    #게시글 링크 찾기
    address.extend([x.get_attribute('href') for x in table.find_elements_by_tag_name('a')])

    #게시글 작성 시간 찾기
    times.extend([x.text for x in table.find_elements_by_class_name('tah.p10.gray03') if len(x.text)>10])

    #게시글 내용 찾기

    for addr in address:
        driver.get(addr)
        contents.append(driver.find_element_by_id('body').text.replace(","," "))

    now= datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    board_contents = pd.DataFrame(
        {'title':title,'contents':contents,'time':times}
    )
    board_contents.to_csv('./categories/'+folder_name+'/'+company_info_lists[1]+now+'.csv', encoding='utf-8')
    driver.close()


In [3]:
# #파일경로
# file_paths= glob('E:/big12/python-project/note/categories/*.csv')
# file_path_root ='E:/big12/python-project/note/categories/'

def folder_file_separator(file_paths):
    
    #업종별 csv가 저장되어있는 경로를 입력해주세요(루트까지만)
    file_path_root ='E:/big12/python-project/note/categories/'
#파일이름을 기초로 찾아보기.
    for file_path in tqdm(file_paths):
        #종목 폴더이름
        folder_name=[]
        folder_name = str(file_path.split('\\')[1].split('.')[0])
        try:
            if not(os.path.isdir(file_path_root+folder_name)):
                os.makedirs(os.path.join(file_path_root+folder_name))
        except OSError as e:
            if e.errno != errno.EEXIST:
                print("Failed to create directory!!!!!")
                raise
        time.sleep(0.7)

        #종목 회사 별코드
        code_list= [x[1:] for x in np.array(pd.read_csv(file_path)['code'])]
        #종목별회사명
        name_list = [x for x in np.array(pd.read_csv(file_path)['name'])]
        company_info_list=[]
        for code,name in zip(code_list,name_list):
            company_info_list.append([code,name])

        for company_infos in company_info_list:
            print(company_infos,folder_name,'from separator')
            crawl_(company_infos,folder_name)

In [4]:
#파일경로
###4개의 멀티쓰레딩으로 정보를 모을겁니다. 
### 코어의 개수 * 2 하신게 최대 쓰레드라고 보시면 되고요.. 저는 그냥 반정도 욕심내지 않고 하려합니다.
### 멀티쓰레딩에 사용되는 모듈 및 function은 from multiprocessing.pool import ThreadPool입니다.
### author : 편도훈 a.k.a '중원구 민트초코 패밀리사이즈' 입니다.

## 업종별 모음하신 파일 경로를 입력해주세요.
file_paths_all= glob('E:/big12/python-project/note/categories/*.csv')

file_paths1=[]
file_paths2=[]
file_paths3=[]
file_paths4=[]
for i in range(len(file_paths_all)):
    if i%4==0: file_paths1.append(file_paths_all[i])
    elif i%4==1 : file_paths2.append(file_paths_all[i])
    elif i%4==2 : file_paths3.append(file_paths_all[i])
    elif i%4==3 : file_paths4.append(file_paths_all[i])
### starmap은 왠지 모르겠는데 2중리스트를 1중리스트로 풀어서 그 1중리스트를 또 풀어 인자로 전달한다.. .인자만 16개가 되는 대 참 사가
### 일어나기 때문에 한번 더 감싸줘서 3중리스트로 안전하고 꼼꼼하게 우겨넣었더니 되더라
file_paths_separated_list= [[file_paths1[2:3]],[file_paths2[2:3]],[file_paths3[2:3]],[file_paths4[2:3]]]
now1 = time.time()
with ThreadPool(4) as tp:
    tp.starmap(
    folder_file_separator,file_paths_separated_list
    )
now2= time.time()

print('총걸린시간 : '+str(now2-now1))

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

['101730', '조이맥스'] 게임엔터테인먼트 from separator
['225530', '보광산업']['038010', '제일테크노스'] ['076080', '웰크론한텍'] 건설 from separator
 건축제품 from separator
건축자재 from separator
['076080', '웰크론한텍']
['101730', '조이맥스']
['225530', '보광산업']['038010', '제일테크노스']

['044180', 'KD'] 건설 from separator
['004985', '성신양회우'] 건축자재 from separator
['039020', '이건홀딩스'] 건축제품 from separator
['060240', '룽투코리아'] 게임엔터테인먼트 from separator
['044180', 'KD']
['004985', '성신양회우']
['039020', '이건홀딩스']
['060240', '룽투코리아']
['002785', '진흥기업우B']['004980', '성신양회']  건설 from separator
건축자재 from separator
['004980', '성신양회']
['002785', '진흥기업우B']
['005750', '대림B&Co'] 건축제품 from separator
['112040', '위메이드'] 게임엔터테인먼트 from separator
['005750', '대림B&Co']
['112040', '위메이드']
['108670', 'LG하우시스'] 건축자재 from separator
['013700', '까뮤이앤씨'] 건설 from separator
['108670', 'LG하우시스']
['225570', '넷게임즈'] 게임엔터테인먼트 from separator
['013700', '까뮤이앤씨']
['022220', '정산애강'] 건축제품 from separator
['022220', '정산애강']
['225570', '넷게임즈']
['006390', '한일현대시멘트'] 건축자재 from separator



100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [13:20<00:00, 800.23s/it]

['052770', '아이톡시'] 게임엔터테인먼트 from separator


['052770', '아이톡시']
['375500', 'DL이앤씨'] 건설 from separator
['375500', 'DL이앤씨']
['038500', '삼표시멘트'] 건축자재 from separator
['038500', '삼표시멘트']
['056000', 'COWON'] 게임엔터테인먼트 from separator
['006360', 'GS건설'] 건설 from separator
['056000', 'COWON']
['006360', 'GS건설']
['293490', '카카오게임즈'] 게임엔터테인먼트 from separator
['293490', '카카오게임즈']
['002150', '도화엔지니어링'] 건설 from separator
['002150', '도화엔지니어링']
['095660', '네오위즈'] 게임엔터테인먼트 from separator
['095660', '네오위즈']
['047040', '대우건설'] 건설 from separator
['000325', '노루홀딩스우'] 건축자재 from separator
['047040', '대우건설']
['000325', '노루홀딩스우']
['067000', '조이시티'] 게임엔터테인먼트 from separator
['067000', '조이시티']
['007110', '일신석재'] 건축자재 from separator
['097230', '한진중공업'] 건설 from separator
['007110', '일신석재']
['097230', '한진중공업']
['108675', 'LG하우시스우'] 건축자재 from separator
['010400', '우진아이엔에스'] 건설 from separator
['108675', 'LG하우시스우']
['010400', '우진아이엔에스']
['047080', '한빛소프트'] 게임엔터테인먼트 from separator
['047080', '한빛소프트']
['003410', '쌍용양회'] 건축자재 from separator
['007680', '대원'] 건설 from sep

...

['013580', '계룡건설'] 건설 from separator
['013580', '계룡건설']
['003300', '한일홀딩스'] 건축자재 from separator
['003300', '한일홀딩스']
['003075', '코오롱글로벌우'] 건설 from separator
['003075', '코오롱글로벌우']
['000860', '강남제비스코'] 건축자재 from separator
['000860', '강남제비스코']
['101670', '코리아에스이'] 건설 from separator
['101670', '코리아에스이']
['001529', '동양3우B'] 건축자재 from separator
['001529', '동양3우B']
['037440', '희림'] 건설 from separator
['037440', '희림']
['002990', '금호산업'] 건설 from separator
['002990', '금호산업']
['001527', '동양2우B'] 건축자재 from separator
['001527', '동양2우B']
['053690', '한미글로벌'] 건설 from separator
['053690', '한미글로벌']
['344820', '케이씨씨글라스'] 건축자재 from separator
['344820', '케이씨씨글라스']
['004960', '한신공영'] 건설 from separator
['004960', '한신공영']
['035890', '서희건설'] 건설 from separator
['035890', '서희건설']
['006920', '모헨즈'] 건축자재 from separator
['006920', '모헨즈']
['37550K', 'DL이앤씨우'] 건설 from separator
['37550K', 'DL이앤씨우']
['002380', 'KCC'] 건축자재 from separator
['002380', 'KCC']
['294870', 'HDC현대산업개발'] 건설 from separator
['294870', 'HDC현대산업개발']




100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [24:17<00:00, 1457.13s/it]


['039570', 'HDC아이콘트롤스'] 건설 from separator
['039570', 'HDC아이콘트롤스']
['023960', '에쓰씨엔지니어링'] 건설 from separator
['023960', '에쓰씨엔지니어링']
['001880', '대림건설'] 건설 from separator
['001880', '대림건설']
['001840', '이화공영'] 건설 from separator
['001840', '이화공영']
['036190', '금화피에스시'] 건설 from separator
['036190', '금화피에스시']
['000215', 'DL우'] 건설 from separator
['000215', 'DL우']
['054930', '유신'] 건설 from separator
['054930', '유신']
['000210', 'DL'] 건설 from separator
['000210', 'DL']
['005965', '동부건설우'] 건설 from separator
['005965', '동부건설우']
['028100', '동아지질'] 건설 from separator
['028100', '동아지질']
['013360', '일성건설'] 건설 from separator
['013360', '일성건설']
['002410', '범양건영'] 건설 from separator
['002410', '범양건영']
['066620', '국보디자인'] 건설 from separator
['066620', '국보디자인']
['014790', '한라'] 건설 from separator
['014790', '한라']
['001260', '남광토건'] 건설 from separator
['001260', '남광토건']
['005960', '동부건설'] 건설 from separator
['005960', '동부건설']
['026150', '특수건설'] 건설 from separator
['026150', '특수건설']
['001470', '삼부토건'] 건설 from separator

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [28:57<00:00, 1737.33s/it]

총걸린시간 : 1737.3867228031158


In [ ]:
# #파일경로
###개갓다
# file_paths= glob('E:/big12/python-project/note/categories/*.csv')
# file_path_root ='E:/big12/python-project/note/categories/'
# #파일이름을 기초로 찾아보기.
# for file_path in file_paths[1:]:
#     #종목 폴더이름
#     folder_name = str(file_path.split('\\')[1].split('.')[0])
    
#     try:
#         if not(os.path.isdir(file_path_root+folder_name)):
#             os.makedirs(os.path.join(file_path_root+folder_name))
#     except OSError as e:
#         if e.errno != errno.EEXIST:
#             print("Failed to create directory!!!!!")
#             raise
#     time.sleep(0.7)
    
#     #종목 회사 별코드
#     code_list= [x[1:] for x in np.array(pd.read_csv(file_path)['code'])]
#     #종목별회사명
#     name_list = [x for x in np.array(pd.read_csv(file_path)['name'])]
#     company_info_list=[]
#     for code,name in zip(code_list,name_list):
#         company_info_list.append([code,name])
        
#     for company_infos in company_info_list:
#         print(company_infos,folder_name)
#         crawl_(company_infos,folder_name)

In [12]:
now1 =time.time()
time.sleep(1)
now2=time.time()

print('총걸린시간 : '+str(now2-now1))

총걸린시간 : 1.0000858306884766
